### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Lower_Grade_Glioma/GSE15309'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data based on the background information.
if "SAGE" in "!Series_summary" and "gene expression" in "!Series_summary":
    is_gene_available = True

# Identify keys for 'Lower_Grade_Glioma', 'age', and 'gender'.
sample_characteristics = {
    0: ['organ/tissue: bone marrow', 'organ/tissue: brain', ...],
    1: ['tissue preparation: flow-sorted', 'tissue preparation: antibody purified', ...],
    2: ['sex: unknown', 'age: fetal', 'age: 34', 'age: 35 yrs', 'age: 23', 'age: 9', ...],
    3: ['cell type: Stem Proginitor Cells, CD34+, CD38+', 'cell type: Acute myelogenous leukemia', ...],
    4: [nan, 'other information: Normal human astrocyte cells selected from the whole brain of a 22.5 wks old donor grown in the supplied media and harvested at passage 5. Cat# CC-2565, Lot 17657.', ...],
    5: [nan, 'other information: grade one(1) pilocytic astrocytoma', 'other information: Malignant embryonal tumor consistent with ependymoblastoma, ...],
    6: [nan, 'other information: Invasive ductal carcinoma - primary tumor.  Age at discovery 56.5 years, 10 postive lymph nodes, ...]
}

# Assign trait_row, age_row, and gender_row by identifying keys in the sample characteristics dictionary.

# Lower_Grade_Glioma: Assuming its presence can be inferred, map it to "cell type: glioblastoma" and related types.
possible_glioma_values = [
    'cell type: glioblastoma, Lt. Temporal, cerebral cortex',
    'cell type: glioblastoma, cerebral cortex',
    'cell type: glioblastoma,',
    'cell type: Grade II',
    'cell type: primary brain tumor in right foramen of lushka',
    # Other related subtypes...
]
for key, values in sample_characteristics.items():
    if any(value in possible_glioma_values for value in values):
        trait_row = key
        break

# Age: Data available in row 2.
age_row = 2

# Gender: Data available in row 3 with 'sex' as either 'sex: male' or 'sex: female'.
for key, values in sample_characteristics.items():
    if 'sex: male' in values or 'sex: female' in values:
        gender_row = key
        break

# Define conversion functions.
def convert_trait(value):
    if value in possible_glioma_values:
        return 1  # Assume presence of Lower_Grade_Glioma
    elif not value or value is nan:
        return None
    else:
        return 0

def convert_age(value):
    if not value or value is nan:
        return None
    try:
        age = value.split(': ')[-1]
        if 'years' in age or 'yrs' in age or 'yr' in age:
            return float(age.replace('years', '').replace('yrs', '').replace('yr', '').strip())
        return float(age)  # Considering fetal as a continuous variable for simplicity
    except:
        return None

def convert_gender(value):
    if not value or value is nan:
        return None
    gender = value.split(': ')[-1].lower()
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    return None

# Save Cohort Information
save_cohort_info('GSE15309', './preprocessed/Lower_Grade_Glioma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Lower_Grade_Glioma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Lower_Grade_Glioma/trait_data/GSE15309.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
